In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/train_data')

In [ ]:
df

,alg,mor,label
0,",",، إلى,1
1,",",، إلى عند,1
2,", إلا نكتبوها بالفرنسي",، ف كتابتها ب فرونسي,1
3,", إلا نكتبوها بالفرنسي ولا",، ف كتابتها ب فرونسي ولا,1
4,", اللي راهو إيفيكاس , ورايح يجيب",لي ناجح و غدي,1
...,...,...,...
330951,أول ماي بالخميس على,متافقين بلي فرحة البكالوريا مكيناش فرحة,0
330952,أول ماي بالخميس على شحال,متافقين بلي فرحة البكالوريا مكيناش فرحة بحالها,0
330953,أول محرم,يسوتني,0
330954,أول محرم,وقالت,0


In [ ]:
df_mor=df.mor.values

In [ ]:
df_mor

array([' ، إلى ', ' ، إلى عند ', ' ، ف كتابتها ب فرونسي ', ...,
       ' يسوتني ', ' وقالت ', ' ، إلى '], dtype=object)

In [ ]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(df_mor):
    # remove newline characters from each sample as we need to use exclusively as seperators
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/330956 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('./').glob('**/*.txt')]
paths

['text_19.txt',
 'text_12.txt',
 'text_17.txt',
 'text_64.txt',
 'text_20.txt',
 'text_16.txt',
 'text_41.txt',
 'text_35.txt',
 'text_22.txt',
 'text_31.txt',
 'text_57.txt',
 'text_25.txt',
 'text_11.txt',
 'text_47.txt',
 'text_30.txt',
 'text_6.txt',
 'text_33.txt',
 'text_36.txt',
 'text_52.txt',
 'text_62.txt',
 'text_60.txt',
 'text_0.txt',
 'text_10.txt',
 'text_4.txt',
 'text_27.txt',
 'text_63.txt',
 'text_49.txt',
 'text_15.txt',
 'text_13.txt',
 'text_65.txt',
 'text_38.txt',
 'text_18.txt',
 'text_66.txt',
 'text_14.txt',
 'text_34.txt',
 'text_42.txt',
 'text_55.txt',
 'text_23.txt',
 'text_26.txt',
 'text_43.txt',
 'text_32.txt',
 'text_53.txt',
 'text_45.txt',
 'text_61.txt',
 'text_48.txt',
 'text_9.txt',
 'text_58.txt',
 'text_28.txt',
 'text_1.txt',
 'text_2.txt',
 'text_8.txt',
 'text_46.txt',
 'text_44.txt',
 'text_3.txt',
 'text_7.txt',
 'text_40.txt',
 'text_54.txt',
 'text_5.txt',
 'text_59.txt',
 'text_24.txt',
 'text_39.txt',
 'text_51.txt',
 'text_37.txt',
 '

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False,
    

)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [ ]:
import os

In [ ]:
os.mkdir('daribert')

tokenizer.save_model('./daribert', 'daribert')

['./daribert/daribert-vocab.txt']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./daribert')

In [ ]:
tokenizer.tokenize('عندو مشاكل ف سوتنانس و نتوما لازم')

['عندو', 'مشاكل', 'ف', 'سوتنانس', 'و', 'نتوما', 'لازم']